# Шаблон для вычисления метрик

**Автор:** [Ваше имя]  
**Дата:** [Дата]  
**Эксперимент:** [Название эксперимента]  

## Описание

Этот ноутбук предназначен для вычисления метрик качества распознавания текста.

**Метрики:**
- CER (Character Error Rate)
- WER (Word Error Rate)
- Character Accuracy
- Levenshtein Distance

**Требования:**
- Файл с предсказаниями (predictions.csv)
- Ground truth метки (если есть)

## 1. Подключение Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2. Установка зависимостей

In [ ]:
# Клонирование репозитория
!git clone https://github.com/Nadarsa/handwritten-ocr.git
%cd handwritten-ocr

# Установка зависимостей
!pip install -q -r requirements.txt

## 3. Настройка путей и импорты

In [ ]:
import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Добавить src в Python path
sys.path.append('/content/handwritten-ocr/src')

# Импорт модулей проекта
from metrics import calculate_cer, calculate_wer, calculate_char_accuracy, MetricsEvaluator

# Пути
PROJECT_PATH = '/content/drive/MyDrive/Практикум_3_семестр/handwritten-ocr'
RESULTS_PATH = os.path.join(PROJECT_PATH, 'results')

print(f"Путь к результатам: {RESULTS_PATH}")

## 4. Загрузка предсказаний

**ВАЖНО:** Замените на имя вашего эксперимента

In [ ]:
# Имя эксперимента
EXPERIMENT_NAME = 'trocr_hwr200_test'  # ИЗМЕНИТЕ НА ВАШЕ НАЗВАНИЕ

# Путь к predictions.csv
predictions_path = os.path.join(RESULTS_PATH, EXPERIMENT_NAME, 'predictions.csv')

# Загрузка
if os.path.exists(predictions_path):
    df = pd.read_csv(predictions_path)
    print(f"Загружено предсказаний: {len(df)}")
    print("\nПервые 5 строк:")
    print(df.head())
else:
    print(f"Файл не найден: {predictions_path}")
    print("Сначала запустите inference_template.ipynb")

## 5. Загрузка Ground Truth меток

**Если у вас есть файл с истинными метками**

In [ ]:
# Если у вас есть CSV с метками
# labels_path = os.path.join(PROJECT_PATH, 'data/raw/hwr200/labels.csv')
# df_labels = pd.read_csv(labels_path)

# Или создать вручную для демонстрации
# ПРИМЕР: замените на реальные метки
ground_truth = [
    "Пример текста 1",
    "Пример текста 2",
    "Пример текста 3"
    # ... добавьте все метки
]

# Если меток нет, оставьте пустым список
# ground_truth = []

print(f"Загружено ground truth меток: {len(ground_truth)}")

## 6. Вычисление метрик

**ТРЕБУЕТ GROUND TRUTH**

In [ ]:
if ground_truth and len(ground_truth) == len(df):
    # Получить предсказания
    predictions = df['predicted_text'].tolist()
    
    # Вычислить метрики
    evaluator = MetricsEvaluator()
    metrics = evaluator.evaluate(predictions, ground_truth)
    
    # Вывести результаты
    evaluator.print_results()
    
    # Сохранить метрики
    metrics_path = os.path.join(RESULTS_PATH, EXPERIMENT_NAME, 'quality_metrics.json')
    with open(metrics_path, 'w', encoding='utf-8') as f:
        json.dump(metrics, f, ensure_ascii=False, indent=2)
    print(f"\nМетрики сохранены: {metrics_path}")
else:
    print("Ground truth метки отсутствуют или не совпадают по размеру.")
    print("Пропускаем вычисление метрик качества.")
    metrics = None

## 7. Анализ ошибок

**Требует Ground Truth**

In [ ]:
if metrics:
    from Levenshtein import distance as levenshtein_distance
    
    # Вычислить расстояние для каждой пары
    errors = []
    for pred, gt in zip(predictions, ground_truth):
        dist = levenshtein_distance(pred, gt)
        errors.append({
            'predicted': pred,
            'ground_truth': gt,
            'distance': dist,
            'error': dist > 0
        })
    
    df_errors = pd.DataFrame(errors)
    
    # Статистика
    num_errors = df_errors['error'].sum()
    accuracy = 1 - (num_errors / len(df_errors))
    
    print(f"Общая точность (на уровне строк): {accuracy * 100:.2f}%")
    print(f"Количество ошибок: {num_errors} из {len(df_errors)}")
    
    # Показать худшие примеры
    print("\nХудшие примеры (наибольшее расстояние):")
    worst_examples = df_errors.nlargest(5, 'distance')
    for idx, row in worst_examples.iterrows():
        print(f"\nПример {idx + 1}:")
        print(f"  Ground Truth: {row['ground_truth']}")
        print(f"  Predicted:    {row['predicted']}")
        print(f"  Distance:     {row['distance']}")

## 8. Визуализация результатов

In [ ]:
if metrics:
    # График метрик
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # График 1: Метрики
    metric_names = ['CER', 'WER']
    metric_values = [metrics['cer'], metrics['wer']]
    
    axes[0].bar(metric_names, metric_values, color=['#FF6B6B', '#4ECDC4'])
    axes[0].set_ylabel('Значение (%)')
    axes[0].set_title('Метрики ошибок')
    axes[0].set_ylim(0, max(metric_values) * 1.2)
    
    for i, v in enumerate(metric_values):
        axes[0].text(i, v + 1, f'{v:.2f}%', ha='center', fontweight='bold')
    
    # График 2: Точность
    accuracy_value = metrics['char_accuracy']
    axes[1].bar(['Точность (символы)'], [accuracy_value], color='#95E1D3')
    axes[1].set_ylabel('Значение (%)')
    axes[1].set_title('Точность распознавания')
    axes[1].set_ylim(0, 100)
    axes[1].text(0, accuracy_value + 2, f'{accuracy_value:.2f}%', ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Распределение ошибок
    plt.figure(figsize=(10, 5))
    plt.hist(df_errors['distance'], bins=30, edgecolor='black')
    plt.xlabel('Levenshtein Distance')
    plt.ylabel('Частота')
    plt.title('Распределение расстояний редактирования')
    plt.axvline(df_errors['distance'].mean(), color='red', linestyle='--', 
                label=f'Среднее: {df_errors["distance"].mean():.2f}')
    plt.legend()
    plt.show()

## 9. Анализ типов ошибок

In [ ]:
if metrics:
    # Анализ длины текста vs ошибки
    df_errors['gt_length'] = df_errors['ground_truth'].str.len()
    df_errors['pred_length'] = df_errors['predicted'].str.len()
    
    # Корреляция между длиной и ошибками
    plt.figure(figsize=(10, 5))
    plt.scatter(df_errors['gt_length'], df_errors['distance'], alpha=0.5)
    plt.xlabel('Длина ground truth текста (символов)')
    plt.ylabel('Levenshtein Distance')
    plt.title('Зависимость ошибок от длины текста')
    plt.show()
    
    # Средняя ошибка по длине
    length_bins = pd.cut(df_errors['gt_length'], bins=5)
    error_by_length = df_errors.groupby(length_bins)['distance'].mean()
    
    print("\nСредняя ошибка по длине текста:")
    print(error_by_length)

## 10. Объединение метрик качества и производительности

In [ ]:
# Загрузить метрики производительности
performance_path = os.path.join(RESULTS_PATH, EXPERIMENT_NAME, 'performance_metrics.json')

if os.path.exists(performance_path):
    with open(performance_path, 'r', encoding='utf-8') as f:
        performance_metrics = json.load(f)
    
    # Объединить все метрики
    combined_metrics = {
        'experiment_name': EXPERIMENT_NAME,
        'model': performance_metrics['model_name'],
        'device': performance_metrics['device'],
        'num_samples': performance_metrics['num_samples'],
        
        # Метрики качества
        'cer': metrics['cer'] if metrics else None,
        'wer': metrics['wer'] if metrics else None,
        'char_accuracy': metrics['char_accuracy'] if metrics else None,
        
        # Метрики производительности
        'avg_inference_time_ms': performance_metrics['avg_inference_time_ms'],
        'median_inference_time_ms': performance_metrics['median_inference_time_ms'],
    }
    
    # Сохранить объединенные метрики
    combined_path = os.path.join(RESULTS_PATH, EXPERIMENT_NAME, 'combined_metrics.json')
    with open(combined_path, 'w', encoding='utf-8') as f:
        json.dump(combined_metrics, f, ensure_ascii=False, indent=2)
    
    print("Объединенные метрики:")
    print(json.dumps(combined_metrics, indent=2, ensure_ascii=False))
    print(f"\nСохранено: {combined_path}")

## Выводы

**Напишите ваши выводы:**
- Качество распознавания (CER, WER)
- Производительность (время)
- Типичные ошибки модели
- Зависимость ошибок от длины текста
- Рекомендации для улучшения

---

**Следующие шаги:**
1. Сравнить с другими моделями
2. Проанализировать confusion matrix для символов
3. Определить оптимальную модель